In [ ]:
import gym

env = gym.make('CartPole-v0')
print(env.action_space)
print(env.observation_space)

Cartpole er et problem som består i en platform som kan bevege seg horisontalt og en pendel festet til platformen. Spillet går ut på å balansere pendelen over platformen. Observasjonsrommet er posisjonen og hastigheten til platformen, samt vinkelen og vinkelhastigheten til pendelen. Handlingene er et dytt mot venstre eller høyre.

For å lære en strategi for dette problemet kan man bruke Deep Q Learning. Dette er en metode hvor man setter opp et dypt nevralt nettverk hvor inputen er observasjonen og outputen er en handling. Bellmans ligning gir

$Q(S_t, A_t,\theta) \leftarrow Q(S_t, A_t,\theta) + \alpha [R_t + \gamma \max_a Q(S_{t+1}, a,\theta) - Q(S_t,A_t, \theta)]$

Hvor $\theta$ er vektene til Q-nettverket. Dette kan oversettes til problemet å minimere tapsfunksjonen

$L(\theta) = |\underline{R_t + \gamma \max_a Q(S_{t+1}, a,\theta)} - Q(S_t,A_t,\theta)|^2$

Ettersom man her optimaliserer mot en target (den understrekede delen av tapsfunksjonen) som ikke er stasjonær, altså endrer seg etterhvert som man trener nettverket, kan vi ikke garantere at treningen konvergerer. For å få en mer stabil trening vil man gjerne bruke to nettverk, et target nettverk og et treningsnettverk, hvor vektene fra targetnettverket kun endrer seg etter et gitt antall iterasjoner.

In [ ]:
from keras.models import Sequential
from keras.layers import Dense, Activation, Flatten
from keras.optimizers import Adam

input_shape = (1,) + # ?
num_outputs = # ?

model = Sequential()
model.add(Flatten(input_shape=input_shape))
# Definer model
model.add(Dense(env.action_space.n, activation='softmax'))

Biblioteket <code>keras-rl</code> kommer med innebygd støtte for Deep Q nettverk. Her setter vi opp en agent som gitt en model, et environment og en policy (metode for å velge ut handling. Her bruker vi en grådig policy som bare velger den handlingen med størst vekting).

In [ ]:
from rl.agents.dqn import DQNAgent
from rl.policy import EpsGreedyQPolicy
from rl.memory import SequentialMemory

policy = EpsGreedyQPolicy()
memory = SequentialMemory()
dqn = DQNAgent(model=model, nb_actions= , memory=memory, nb_steps_warmup=10, target_model_update=1e-2, policy=policy)

dqn.compile(Adam(lr=1e-3), metrics=['mae'])

dqn.fit(env, nb_steps=5000, verbose=2)

Til slutt kan vi teste agenten vår på problemet ved å kjøre <code>dqn.test()</code>

In [ ]:
dqn.test(env, nb_episodes=5, visualize=True)
env.close()

Dersom du blir ferdig kan du prøve å trene opp en agent på et mer komplisert miljø og nettverk, for eksempel et CNN på miljøet <code>"Breakout-v0"</code>.